In [2]:
import pandas as pd
import pickle
import pymongo

Δεδομένων όλων των ομιλιών, πρέπει να ανιχνεύσουμε ομοιότητες ανά ζεύγη μεταξύ των μελών του κοινοβουλίου. Συγκεκριμένα, πρέπει να βρούμε έναν τρόπο να εξαγάγουμε ένα διάνυσμα χαρακτηριστικών για κάθε μέλος και στη συνέχεια να εκτελέσουμε ομοιότητες ανά ζεύγη για να μπορέσουμε να ανιχνεύσουμε τα top-k ζεύγη με τον υψηλότερο βαθμό ομοιότητας (όπου k είναι μια παράμετρος).

In [3]:
# CONNECT TO DATABASE
mongo_client = pymongo.MongoClient("mongodb://localhost:27017/")
client = mongo_client["GreekParliamentProceedings"]
index = client["InvertedIndex"]
database = client["Database"]

In [4]:
# READ KEYWORDS BY MEMBER
with open('results/member_keywords.pickle', 'rb') as handle:
    keywords_by_member = pickle.load(handle)
member_names = keywords_by_member.keys()
keywords = [list(keywords_by_member[member].keys()) for member in member_names]

### Διάνυσμα χαρακτηριστικών για κάθε μέλος

In [5]:
#CREATE A FEATURE VECTOR FOR EACH MEMBER
member_names = keywords_by_member.keys()
keywords = [list(keywords_by_member[member].keys()) for member in member_names]
genders = []
political_parties = []
member_roles = []
regions = []
governments = []
def get_attributes_by_member(members, collection):
        """Creates an attribute vector for each member in a list of members.
        The attributes are gathered from a MongoDB collection.
        Each member has the following attributes: 
            name, gender, political parties, roles, regions, governments, keywords

        Parameters
        ----------
        members : list
            The names of the members
        
        collection: pymongo.collection.Collection
            A collection which includes information about the member attributes.
            The collection documents must include the following fields:
            'member_name', 'roles', 'member_gender', 'government', 'political_party', 'member_region'

        """
        
        genders = []
        political_parties = []
        member_roles = []
        regions = []
        governments = []
        for member in members:
                pipeline = [{'$match' : {'member_name':member}},
                            {'$facet': {
                                'groupByRoles':[{'$group':{'_id':'$roles'}}],
                                'groupByGender':[{'$group':{'_id':'$member_gender'}}],
                                'groupByGovernment':[{'$group':{'_id':'$government'}}],
                                'groupByParty':[{'$group':{'_id':'$political_party'}}],
                                'groupByRegion':[{'$group':{'_id':'$member_region'}}]
                            }
                                        }
                        ]
                r = list(collection.aggregate(pipeline))
                genders.append(r[0]['groupByGender'][0]['_id'])
                regions.append([r[0]['groupByRegion'][i]['_id'] for i in range(len(r[0]['groupByRegion']))])
                member_roles.append([r[0]['groupByRoles'][i]['_id'] for i in range(len(r[0]['groupByRoles']))])
                governments.append([r[0]['groupByGovernment'][i]['_id'] for i in range(len(r[0]['groupByGovernment']))])
                political_parties.append([r[0]['groupByParty'][i]['_id'] for i in range(len(r[0]['groupByParty']))])
                
        return genders, regions, political_parties, member_roles, governments

In [6]:
genders, regions, political_parties, member_roles, governments = get_attributes_by_member(member_names, database)

In [17]:

member_names = list(member_names)
feature_vectors = []
for i in range(len(keywords)):
    feature_vectors.append([])
    feature_vectors[i].append(member_names[i].split()[0])
    feature_vectors[i].append(genders[i])
    feature_vectors[i].extend(regions[i][j] for j in range(len(regions[i])))
    feature_vectors[i].extend(member_roles[i][j] for j in range(len(member_roles[i])))
    feature_vectors[i].extend(governments[i][j] for j in range(len(governments[i])))
    feature_vectors[i].extend(political_parties[i][j] for j in range(len(political_parties[i])))
    feature_vectors[i].extend(keywords[i])

In [26]:
#with open('feature_vectors.pickle', 'wb') as handle:
#   pickle.dump(feature_vectors, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [33]:
import pickle
with open('results/feature_vectors.pickle', 'rb') as handle:
    feature_vectors = pickle.load(handle)
print(feature_vectors[1])
from top_k_pairs import Top_k_Pairs as topk
top = topk()
pairs = top.get_pairs(10, feature_vectors, threshold=0.5)

['αβραμακης', 'male', 'σερρων', "['βουλευτης']", "['μητσοτακη κυριακου(08/07/2019-28/07/2020)']", 'συνασπισμος ριζοσπαστικης αριστερας', 'συριζ', 'νομοσχεδ', 'αγροτ', 'διαταξ', 'αυξης', 'δικαιωμ']


In [97]:
keys = sorted(list(pairs))
print(keys)
k = 10
i = len(keys)-1
topk = {}
while(i>0 and len(topk)!=k):
    limit = k
    values = [ pairs[keys[i]][j] for j in range(len(pairs[keys[i]])) ]
    if(len(values)>=k):
        topk[round(keys[i],3)]=values[:limit]
    else:
        topk[round(keys[i],3)]=values
        limit = limit - len(values)
        i -= 1
        
print(topk)

[0.5128205128205128, 0.5135135135135135, 0.5185185185185185, 0.52, 0.5217391304347826, 0.525, 0.5263157894736842, 0.5294117647058824, 0.5333333333333333, 0.5357142857142857, 0.5384615384615384, 0.5454545454545454, 0.55, 0.5555555555555556, 0.56, 0.5675675675675675, 0.5789473684210527, 0.5833333333333334, 0.5909090909090909, 0.5945945945945946, 0.6, 0.6060606060606061, 0.6363636363636364, 0.6666666666666666, 0.7142857142857143]
{0.714: [[238, 1487]], 0.667: [[1264, 495]], 0.636: [[108, 406]], 0.606: [[1216, 95]], 0.6: [[624, 715]], 0.595: [[985, 108]], 0.591: [[1026, 229]], 0.583: [[33, 1479], [826, 495]], 0.579: [[612, 1253]], 0.568: [[185, 1010]]}


In [105]:
for key in topk.keys():
    for pair in topk[key]:
        v1 = set(feature_vectors[pair[0]])
        v2 = set(feature_vectors[pair[1]])
        member_names=list(member_names)
        print(f"Members {member_names[pair[0]]} and {member_names[pair[1]]} have a jaccard similarity of {key}")
        print(v1.intersection(v2))
        print("\n")

Members γεωργιαδης κωνσταντινου αθανασιος and χατζηιωαννιδης λαζαρου βασιλειος have a jaccard similarity of 0.714
{"['βουλευτης']", 'πανελληνιο σοσιαλιστικο κινημα', 'male', 'ημαθιας', "['σημιτη κωνσταντινου(25/09/1996-13/04/2000)']"}


Members σπανακης νικολαου βασιλειος-πετρος and καλλιανος δημητριου ιωαννης have a jaccard similarity of 0.667
{"['βουλευτης']", 'συριζ', 'νομοσχεδ', 'νεα δημοκρατια', "β3' νοτιου τομεα αθηνων", "['μητσοτακη κυριακου(08/07/2019-28/07/2020)']", 'νε δημοκρατ ', 'male'}


Members αργυρης ιωαννη ευαγγελος and ζηση αθανασιου ροδουλα have a jaccard similarity of 0.636
{'λαϊκου', "['βουλευτης']", 'κομμουνιστ', "['καραμανλη α. κωνσταντινου(19/09/2007-07/10/2009)']", "['παπανδρεου α. γεωργιου(06/10/2009-11/11/2011)']", "['παπαδημου λουκα δ.(11/11/2011-17/05/2012)']", "['σημιτη κωνσταντινου(13/04/2000-10/03/2004)']", 'πανελληνιο σοσιαλιστικο κινημα', 'νε δημοκρατ ', "['σημιτη κωνσταντινου(25/09/1996-13/04/2000)']", "['παπανδρεου ανδρεα(13/10/1993-22/01/1996)']", 